In [1]:
import os
import urllib
import shutil
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import TensorFlow

In [2]:
ws = Workspace.from_config()

In [3]:
ws

Workspace.create(name='mlopsdev', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops')

In [4]:
script_folder = './tf-mnist'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='tf-mnist')

In [5]:
from azureml.core.dataset import Dataset

web_paths = [
            'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path=web_paths)

In [6]:
dataset = dataset.register(workspace=ws,
                           name='mnist dataset',
                           description='training and test dataset',
                           create_new_version=True)

# list the files referenced by dataset
dataset.to_path()

['/http/yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

In [7]:
cluster_name = "cpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target


In [10]:
script_params = {
    '--data-folder': dataset.as_named_input('mnist').as_mount(),
    '--batch-size': 50,
    '--first-layer-neurons': 300,
    '--second-layer-neurons': 100,
    '--learning-rate': 0.01
}

est = TensorFlow(source_directory=script_folder,
                 entry_script='tf_mnist.py',
                 script_params=script_params,
                 compute_target=compute_target,
                 use_gpu=True,
                 pip_packages=['azureml-dataprep[pandas,fuse]'])

WARNING - framework_version is not specified, defaulting to version 1.13.
WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['azureml-dataprep']. We cannot guarantee image build will succeed.


In [ ]:
run = exp.submit(est)
run.wait_for_completion(show_output=True)

RunId: tf-mnist_1594509148_14868b1e
Web View: https://ml.azure.com/experiments/tf-mnist/runs/tf-mnist_1594509148_14868b1e?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev

Streaming azureml-logs/20_image_build_log.txt

2020/07/11 23:12:37 Downloading source code...
2020/07/11 23:12:38 Finished downloading source code
2020/07/11 23:12:38 Creating Docker network: acb_default_network, driver: 'bridge'
2020/07/11 23:12:39 Successfully set up Docker network: acb_default_network
2020/07/11 23:12:39 Setting up Docker configuration...
2020/07/11 23:12:40 Successfully set up Docker configuration
2020/07/11 23:12:40 Logging in to registry: mlopsdev82752047.azurecr.io
2020/07/11 23:12:41 Successfully logged into mlopsdev82752047.azurecr.io
2020/07/11 23:12:41 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/07/11 23:12:41 Scanning for dependencies...
2020/07/11 23:12:41 Successfully scann

In [ ]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='tf-dnn-mnist', 
                           model_path='outputs/model',
                           model_framework=Model.Framework.TENSORFLOW,
                           model_framework_version='1.13.0',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))

In [ ]:
# Create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

In [ ]:
from azureml.core.runconfig import MpiConfiguration
from azureml.train.dnn import TensorFlow

# Tensorflow constructor
estimator= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='script.py',
                      node_count=2,
                      process_count_per_node=1,
                      distributed_training=MpiConfiguration(),
                      framework_version='1.13',
                      use_gpu=True,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

In [ ]:
from azureml.train.dnn import TensorFlow

distributed_training = TensorflowConfiguration()
distributed_training.worker_count = 2

# Tensorflow constructor
tf_est= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='script.py',
                      node_count=2,
                      process_count_per_node=1,
                      distributed_training=distributed_training,
                      use_gpu=True,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

# submit the TensorFlow job
run = exp.submit(tf_est)

In [ ]:
TF_CONFIG='{
    "cluster": {
        "ps": ["host0:2222", "host1:2222"],
        "worker": ["host2:2222", "host3:2222", "host4:2222"],
    },
    "task": {"type": "ps", "index": 0},
    "environment": "cloud"
}'

In [ ]:
import os, json
import tensorflow as tf

tf_config = os.environ.get('TF_CONFIG')
if not tf_config or tf_config == "":
    raise ValueError("TF_CONFIG not found.")
tf_config_json = json.loads(tf_config)
cluster_spec = tf.train.ClusterSpec(cluster)

In [ ]:
service = Model.deploy(ws, "tensorflow-web-service", [model])